In [1]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [49]:
import numpy as np
import gdal
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [50]:
inpRaster = 'Composite_fi_clip.tif' 
outRaster = 'classified_fijup.tif' 
df = pd.read_csv('Classifi_FI_comp.csv')

In [51]:
df.head()

,Band_1,Band_2,Band_3,Band_4,Band_5,Band_6,Band_7,Band_8,Band_9,Band_10,Band_11,ClassID,Class_name
0,0.802243,0.511247,-11.121046,-1.187986,-0.217070,46,1.045,0.114,0.089072,0.028547,0.008142,1,ASK
1,0.783197,0.534009,-11.905989,0.034768,-0.557473,54,1.049,0.117,0.090146,0.029851,0.008729,1,ASK
2,0.771204,0.547740,-7.164552,0.445399,-0.423039,54,1.027,0.126,0.087076,0.028418,0.005288,1,ASK
3,0.814251,0.496232,-5.439240,-1.435103,-0.324791,59,1.005,0.115,0.085156,0.031047,0.008023,1,ASK
4,0.846771,0.452534,-4.033033,2.218574,-0.247315,53,1.020,0.122,0.082730,0.032016,0.004915,1,ASK


In [52]:
ds = gdal.Open (inpRaster, gdal.GA_ReadOnly)

In [53]:
rows = ds.RasterYSize
cols = ds.RasterXSize
bands = ds.RasterCount
geo_transform = ds.GetGeoTransform()
projection = ds.GetProjectionRef()

In [54]:
#print(rows,cols)
#print(bands)

In [55]:
array = ds.ReadAsArray()
ds = None

In [56]:
#print(array)

In [57]:
array = np.stack(array,axis=2)
array = np.reshape(array, [rows*cols,bands])

In [58]:
print(array[0])

[ 7.5549459e-01  5.6508839e-01 -8.9468546e+00 -1.3432292e+00
 -5.6897938e-02  5.8000000e+01  1.0076194e+00  1.2162700e-01
  8.5740194e-02  2.8904591e-02  7.3349616e-03]


In [59]:
test = pd.DataFrame(array, dtype='float32')
del array

In [60]:
data = df[['Band_1','Band_2','Band_3','Band_4','Band_5','Band_6','Band_7','Band_8','Band_9','Band_10','Band_11']]
lebel = df['ClassID']
del df

In [61]:
clf = lgb.LGBMClassifier(n_estimators=50,n_jobs=-1)
clf.fit(data,lebel)
del data
del lebel

In [62]:
y_pred = clf.predict(test)

In [63]:
del test

In [64]:
classification = y_pred.reshape((rows,cols))
del y_pred

In [65]:
def createGeotiff(outRaster, data, geo_transform, projection):
    # Create a GeoTIFF file with the given data
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data.shape
    rasterDS = driver.Create(outRaster, cols, rows, 1, gdal.GDT_Int32)
    rasterDS.SetGeoTransform(geo_transform)
    rasterDS.SetProjection(projection)
    band = rasterDS.GetRasterBand(1)
    band.WriteArray(data)
    rasterDS = None

In [66]:
createGeotiff(outRaster,classification,geo_transform,projection)